In [1]:
import collections
import os
import datasets
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
import tqdm

os.environ["HF_DATASETS_CACHE"] = r"./data"

In [46]:
train_data, test_data = datasets.load_dataset("imdb", split=["train", "test"])

In [4]:
train_data, train_data.features

(Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }),
 {'text': Value(dtype='string', id=None),
  'label': ClassLabel(names=['neg', 'pos'], id=None)})

In [47]:
train_data = train_data.select(range(5000))
test_data = test_data.select(range(5000))

In [48]:
nlp = spacy.load("en_core_web_sm")

def tokenize_text(data, tokenizer, max_length):
    tokens = [token.text for token in tokenizer(data["text"])][:max_length]
    return {"tokens": tokens}

In [49]:
max_length = 256

train_data = train_data.map(
    tokenize_text, fn_kwargs={"tokenizer": nlp, "max_length": max_length}
)
test_data = test_data.map(
    tokenize_text, fn_kwargs={"tokenizer": nlp, "max_length": max_length}
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [50]:
train_valid_data = train_data.train_test_split(test_size=0.2)
train_data = train_valid_data["train"]
valid_data = train_valid_data["test"]